In [3]:
import os
import dask
import yaml
import graphviz
from pathlib import Path
from datetime import datetime as dt
from kelloggrs.process import *

In [5]:
in_path = Path("../tests/data/sec-sample-pdf")

In [20]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:63669 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 17.18 GB


In [12]:
%%time
for image_file in list((in_path / "9999999997-12-008041-index").glob("*.png")):
    process_image(image_file)

CPU times: user 3.29 s, sys: 226 ms, total: 3.51 s
Wall time: 31.8 s


In [21]:
process_tasks = []
for image_file in list((in_path / "9999999997-12-008041-index").glob("*.png")):
    process_tasks.append(dask.delayed(process_image)(image_file))

In [22]:
%%time
result = dask.compute(process_tasks)

CPU times: user 357 ms, sys: 66.5 ms, total: 423 ms
Wall time: 8.94 s


In [24]:
client.close()